# Rubin (DP0) & Roman (Troxel+23) images

Contact: Melissa Graham

**Based on a tutorial by Chien-Hao Lin.**

Date: Mon Nov 14 2024

RSP Image: Weekly 2024_42

Goal: Interact with the Rubin & Roman simulated images and perform a uniform image processing task (object detection/deblending/measurement) on both. This demonstrates how the space images can help with deblending.

## Introduction

Space-based images have much higher resolution.
Stars and galaxies that are very close together or even overlapping (blended) due to chance
alignments along the line-of-sight can be better distinguished in higher resolution images.
So can actual galaxy mergers in close physical proximity, though this is less common.

It is possible to use the locations of physically distinct objects (deblended objects)
from higher resolution images to make more accurate photometry measurements.
In cases where the higher and lower resolution images are obtained in the same filters,
and with similar depths, it makes sense just to use the higher resolution images alone.

However, Rubin will obtain data in optical filters and Roman in infrared filters.
In this case, using the higher-resolution infrared images to determine the number and
location of distinct objects, and then make photometric measurements in Rubin's
optical-range images, can improve the optical photometric measurements.

Roman DC2 Simulated Images and Catalogs at IRSA IPAC:<br>
https://irsa.ipac.caltech.edu/data/theory/Roman/Troxel2023/overview.html

Troxel et al. (2023):<br>
https://academic.oup.com/mnras/article/522/2/2801/7076879?login=false

## Set up

In [ ]:
import os
import warnings
warnings.filterwarnings('ignore')
import pickle
import numpy as np
import galsim
import importlib
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse, Circle
from matplotlib.collections import PatchCollection
%matplotlib inline
%config InlineBackend.figure_formats = ['svg']
import lsst.geom
import astropy.units as u
from astropy.wcs import WCS
import lsst.meas.algorithms as measAlg
import lsst.afw.math as afwMath
import lsst.afw.geom as afwGeom
import lsst.afw.detection as afwDet
import lsst.afw.table as afwTable
import lsst.afw.image as afwImage
from lsst.meas.algorithms.detection import SourceDetectionTask
from lsst.meas.deblender import SourceDeblendTask
from lsst.meas.base import SingleFrameMeasurementTask
from lsst.daf.butler import Butler as dafButler

In [ ]:
from astropy.visualization import MinMaxInterval, PercentileInterval, ZScaleInterval, simple_norm
def plot(im_, interval='zscale', stretch='linear', title=None, fn=None, xlabel=None, ylabel=None, colorbar=True, cmap='jet', dpi=300, show=False, **kwargs):
    if isinstance(im_, galsim.Image):
        im_ = im_.array
    
    if interval=='zscale':
        interval_ = ZScaleInterval()
    if interval=='minmax':
        interval_ = MinMaxInterval()
    if interval=='percentile':
        interval_ = PercentileInterval()
        
    vmin, vmax = interval_.get_limits(im_)
    norm = simple_norm(im_, stretch=stretch, min_cut=vmin, max_cut=vmax)
    f = plt.imshow(im_, norm=norm, origin='lower', cmap=cmap,  **kwargs)
    cb = plt.colorbar()
    
    if not colorbar:
        cb.remove()
    
    if xlabel is not None:
        plt.xlabel(xlabel)
    if ylabel is not None:
        plt.ylabel(ylabel)
    if title is not None:
        plt.title(title)
    if fn is not None:
        plt.savefig(fn, dpi=dpi)
    if show:
        plt.show()

    return f

initiate butler for Rubin images

In [ ]:
butler = dafButler('dp02', collections='2.2i/runs/DP0.2')

setting up path. For Roman images: four filters for one patch of deeply coadded Roman images have been stored in the shared space in the /project directory.

In [ ]:
lsst_bands = ['r', 'i', 'z', 'y', 'g', 'u']
roman_bands = ['Y106', 'J129', 'H158', 'F184']
roman_bands_i = [ band[0] for band in roman_bands]
roman_fn_dict = {band: '/project/melissagraham2/troxel2023/dc2_%s_54.24_-38.3.fits'%band for band in roman_bands}
roman_fn_psf_dict = {band: '/project/plazas/troxel2023/psf/coadd/dc2_%s_54.24_-38.3_psf.fits'%band for band in roman_bands}

Define `ra` and `dec`, the central coordinates of interest.

Define the scale, in arcseconds per pixel, of Rubin and Roman images.

Define the stamp size to use when visualizing the images (i.e., the cutout size), in Rubin pixels; then use `stampsize / scale_ratio` as the extent when visualizing Roman images.

In [ ]:
ra, dec = 54.28, -38.30 #center
rubin_scale = 0.2
roman_scale = 0.0575
stampsize = 150
scale_ratio = rubin_scale/roman_scale

## Rubin image
Define a function to retrive the `deepCoadd` patch id and return the cutout

In [ ]:
def get_rubin_img(ra, dec, butler, butlerDataset, size, band):
    skymap = butler.get('skyMap')
    radec =  lsst.geom.SpherePoint(ra, dec, lsst.geom.degrees)
    tract = skymap.findTract(radec).tract_id
    patch = skymap.findTract(radec).findPatch(radec).getSequentialIndex()
    dataId = {"tract": tract, "patch": patch, "band": band}
    
    full_patch = butler.get(butlerDataset, dataId=dataId)
    cutout_extent = lsst.geom.ExtentI(size, size)
    exp = full_patch.getCutout(radec, cutout_extent)
    return exp

In [ ]:
imageDict_rubin = {band[0]:get_rubin_img(ra, dec, butler, 'deepCoadd', size=stampsize, band=band) for band in lsst_bands}

Display the rubin images in 6 bands

In [ ]:
for band, img in imageDict_rubin.items():
    bbox = img.getBBox()
    extent = (bbox.getBeginX(),bbox.getEndX(),bbox.getBeginY(),bbox.getEndY())
    fig, ax = plt.subplots(figsize=(8,8), dpi=300)
    plot(img.getMaskedImage().getImage().array, cmap='gray', colorbar=False, extent=extent, title=band)

## Roman image

Load and display a small cutout from each of the four roman bands.

> **Warnings:** Below, the warnings about unreadable mask extensions can be disregarded for the purposes of this tutorial, but generally when using the LSST Science Pipelines with non-Rubin data, all warnings should be followed up and third-party data sets might need to be reformatted to work properly.
In this case the images have four extensions: SCI, WHT, CTX, ERR.
But the `readFits` function expects MASK and IMAGE extensions.

In [ ]:
def get_roman_image(ra, dec, fn_img, size, fn_psf):
    radec = lsst.geom.SpherePoint(ra, dec, lsst.geom.degrees)
    full_patch = lsst.afw.image.ExposureF.readFits(fn_img)
    if fn_psf:
        psf = measAlg.KernelPsf(afwMath.FixedKernel(afwImage.ImageD(fn_psf)))
        full_patch.setPsf(psf)
    cutout_extent = lsst.geom.ExtentI(size, size)
    cutout_extent = lsst.geom.ExtentI(size, size)
    exp = full_patch.getCutout(radec, cutout_extent)
    return exp

In [ ]:
imageDict_roman = {band[0]:get_roman_image(ra, dec, fn_img=roman_fn_dict[band], fn_psf=roman_fn_psf_dict[band], size=stampsize*scale_ratio) for band in roman_bands}

In [ ]:
for band, img in imageDict_roman.items():
    bbox = img.getBBox()
    extent = (bbox.getBeginX(),bbox.getEndX(),bbox.getBeginY(),bbox.getEndY())
    fig, ax = plt.subplots(figsize=(8,8), dpi=300)
    plot(img.getMaskedImage().getImage().array, cmap='gray', colorbar=False, extent=extent, title=band)

## Image processing

Create a single-band image processing task that works with both rubin and roman images. This includes object detection, single band deblender, and source measurement. 

Let's start by setting up the configuration.

In [ ]:
config_detection = SourceDetectionTask.ConfigClass()
config_detection.thresholdValue = 5
config_detection.thresholdType = "stdev"
config_deblend = SourceDeblendTask.ConfigClass()
config_meas = SingleFrameMeasurementTask.ConfigClass() 
config_deblend.propagateAllPeaks = True
config_deblend.maskPlanes=[]

Define the schema.

In [ ]:
schema = afwTable.SourceTable.makeMinimalSchema()
raerr = schema.addField("coord_raErr", type="F")
decerr = schema.addField("coord_decErr", type="F")

Define the image processing task.

In [ ]:
detectionTask = SourceDetectionTask(schema=schema, config=config_detection)
sourceDeblendTask = SourceDeblendTask(schema=schema, config=config_deblend)
measureTask = SingleFrameMeasurementTask(schema=schema, config=config_meas)

Run the processing task on rubin `r` band image. The outputs are in `detections` and `sources`.


`detections`: peaks and footprint of the detected sources

`sources`: source catalog that includes source info and measurement results as defined in the schema.

In [ ]:
exp_r = imageDict_rubin['r']
tab_r = afwTable.SourceTable.make(schema)
detections_r = detectionTask.run(tab_r, exp_r, doSmooth=True, sigma=None)
sources_r = detections_r.sources
sourceDeblendTask.run(exp_r, sources_r)
measureTask.measure(sources_r, exp_r)

Run the processing task on roman `H` band image.

In [ ]:
exp_H = imageDict_roman['H']
tab_H = afwTable.SourceTable.make(schema)
detections_H = detectionTask.run(tab_H, exp_H, doSmooth=True, sigma=None)
sources_H = detections_H.sources
sourceDeblendTask.run(exp_H, sources_H)
measureTask.measure(sources_H, exp_H)

## plot sources and peaks

In [ ]:
#define bbox and figure
bbox = exp_r.getBBox()
extent = (bbox.getBeginX(),bbox.getEndX(),bbox.getBeginY(),bbox.getEndY())
fig, ax = plt.subplots(figsize=(8,8), dpi=300)

#draw the raw img
plot(exp_r.getMaskedImage().getImage().array, cmap='gray', colorbar=False, extent=extent, title='r')

#draw peaks
px=[]
py=[]
for sr in sources_r: #iterate over the sources and get peak info
    fp = sr.getFootprint()
    for pp in fp.getPeaks():
        px.append(pp.getFx())
        py.append(pp.getFy())
plt.scatter(px, py, c='#142c8c', marker='+', linewidths=0.8)  


#draw ellipses with measurement results
#sources['deblend_nChild']==0  -> all the children of blends + isolated object = all peaks 
flag = (sources_r['deblend_nChild']>0) | sources_r['base_PixelFlags_flag']
sources = sources_r[~flag]
x = sources['base_SdssCentroid_x']
y = sources['base_SdssCentroid_y']
axes = [ afwGeom.ellipses.Axes(s.getShape())  for s in sources] 

size_scale = 1.0/rubin_scale
ellipses = [Ellipse( (x[i], y[i]), 
                    width  =axes[i].getA()*size_scale,
                    height =axes[i].getB()*size_scale, 
                    angle  =np.rad2deg(axes[i].getTheta() ) ) for i in range(len(x))]
collection = PatchCollection(ellipses, edgecolor='r', facecolor='None')
ax.add_collection(collection)

plt.tight_layout()
plt.show()


In [ ]:
#define bbox and figure
bbox = exp_H.getBBox()
extent = (bbox.getBeginX(),bbox.getEndX(),bbox.getBeginY(),bbox.getEndY())
fig, ax = plt.subplots(figsize=(8,8), dpi=300)

#draw img
plot(exp_H.getMaskedImage().getImage().array, cmap='gray', colorbar=False, extent=extent, title='H')

#draw peaks
px=[]
py=[]
for sr in sources_H: #iterate over the sources and get peak info
    fp = sr.getFootprint()
    for pp in fp.getPeaks():
        px.append(pp.getFx())
        py.append(pp.getFy())
plt.scatter(px, py, c='#142c8c', marker='+', linewidths=0.8)  


#draw ellipses with measurement results
#sources['deblend_nChild']==0  -> all the children of blends + isolated object = all peaks 
flag = (sources_H['deblend_nChild']>0) | sources_H['base_PixelFlags_flag']
sources = sources_H[~flag]
x = sources['base_SdssCentroid_x']
y = sources['base_SdssCentroid_y']

axes = [ afwGeom.ellipses.Axes(s.getShape())  for s in sources]

size_scale = 0.5/roman_scale
ellipses = [Ellipse( (x[i], y[i]), 
                    width  =axes[i].getA()*size_scale,
                    height =axes[i].getB()*size_scale, 
                    angle  =np.rad2deg(axes[i].getTheta() ) ) for i in range(len(x))]
collection = PatchCollection(ellipses, edgecolor='r', facecolor='None')
ax.add_collection(collection)

plt.tight_layout()
plt.show()